In [ ]:
import pandas as pd

In [ ]:
data =  pd.read_csv("channel_data.csv")
data.head()

In [ ]:
data.info()

In [ ]:
#fillings na values with empty string
data["description"] = data["description"].fillna("")

## LLM parser

For eg. :
The first description is 
```
data['description'][0] = "Rasam Powder: https://youtu.be/YIWBthqArCM?t=59&si=zpnugmEpzvQkN5mb\n\nFull written recipe Rasam Vada\n\nPrep time: 15-20 minutes (excluding soaking time)\nCooking time: 25-30 minutes\nServes: 4-5 people\n\nIngredients:\nRasam\nTUR DAL | तूर दाल ¼ CUP (SOAKED)\nWATER | पानी AS REQUIRED \nTURMERIC POWDER | हल्दी पाउडर ¼ TSP\nOIL | तेल 2 TBSP\nMUSTARD SEED | राई ½ TSP\nCURRY LEAVES | कड़ी पत्ता 12-15 NOS. \nGREEN CHILLI | हरी मिर्च 2 NOS. (SLIT)\nRED CHILLI | लाल मिर्च 4 NOS. \nGARLIC | लहसुन 6 CLOVES (ROUGHLY CHOPPED)\nASAFOETIDA | हींग ¼ TSP\nTOMATO | टमाटर 2 NOS. (CHOPPED)\nCORIANDER STEMS | धनिए के डंठल A HANDFUL (CHOPPED)\nSALT | नमक TO TASTE\nTURMERIC POWDER | हल्दी पाउडर ¼ TSP\nTAMARIND | इमली A LARGE LEMON SIZED BALL. (SOAKED)\nJAGGERY | गुड़ 1 TBSP\nHOT WATER | गरम पानी AS REQUIRED \nRASAM POWDER | रसम पाउडर 3 TBSP\nBLACK PEPPER | काली मिर्च A LARGE PINCH (FRESHLY CRUSHED)\nFRESH CORIANDER | हरा धनिया A HANDFUL (ROUGHLY CHOPPED)\n\nVada\nURAD DAL | उरद दाल 1 CUP\nCOLD WATER | ठंडा पानी AS REQUIRED \nSALT | नमक TO TASTE\nCURRY LEAVES | कड़ी पत्ता 10-12 NOS. (CHOPPED)\nGREEN CHILLI | हरी मिर्च 2 NOS. (CHOPPED)\nGINGER | अदरक 1 INCH (CHOPPED)\nBLACK PEPPER | काली मिर्च A PINCH (FRESHLY CRUSHED)\nASAFOETIDA | हींग ¼ TSP\n Method:\nAdd the washed & soaked tur dal in a pressure cooker along with enough water & turmeric powder, put the lid & pressure cook the dal for 3-4 whistles over high flame.\nOnce cooked let the cooker depressurize naturally & then take off the lid, whisk the dal well using a whisk, make sure it should become completely smooth & watery.\nSet a stock pot over high heat & add oil into it, once the oil gets hot add in the red chillies, mustard seeds, curry leaves, green chilli & garlic, stir well & cook briefly.\nFurther add asafoetida, tomato, salt & coriander stem, stir well & cook until the tomatoes get mushy.\nAdd turmeric powder & tamarind pulp along with jaggery & hot water, stir & cook for 3-4 minutes until the raw smell of the tamarind is gone.\nThen add the cooked tur dal & hot water to adjust the consistency, rasam is always watery, it is not as thick as sambar.\nLet the rasam simmer for 7-8 minutes, if it thickens up then add more hot water to it.\nOnce you have let the rasam simmer, add rasam powder to a bowl along with a few tablespoons of water, stir well until no lumps remain & add the mixture to the rasam & stir well.\nTaste & adjust the salt at this stage & add crushed black peppercorns along with fresh coriander, stir & cook for 2-3 minutes, the rasam is ready. \nWash the urad dal & soak it for 3-4 hours, once the dal is soaked, drain the excess water & add the soaked urad dal into a mixer grinder jar & grind the dal coarsely  using minimal water (it should be ice cold), the texture of the dal should neither be completely smooth nor extremely coarse.\nFurther transfer the dal into a bowl & whisk it for 3-4 minutes using your hand or a whisk until it gets light & fluffy.\nTo check if the batter is ready, add a small portion of it in a bowl of water, if the batter floats, then you can continue with the addition of further ingredients.\nFurther add salt, ginger, green chilli, asafoetida, curry leaves & mix it very gently.\nTo fry the vadas, heat the oil in a kadhai until it gets moderately hot or 170 C.\nTo shape the vadas dip your hand in cold water & scoop out a small bite sized portion of the batter from the bowl & carefully drop it into the hot oil, add other portions similar & then immediately spash hot oil over the vadas so that they puff up & then fry over medium flame while flipping it at regular intervals until light golden brown.\nOnce the vadas get golden brown & crisp, remove them using a spider & transfer them into a sieve, the vadas are ready.\nTo serve the rasam vada, add a few vadas in a bowl & pour hot rasam over it, you can also directly add the vadas in the rasam after frying. \n\n\n#YFL #SanjyotKeer #RasamVada\n\nThe Music I use in All my videos - https://bit.ly/3CdPbSc\n(Best for independent creators)\n\nFollow us on all platforms: \nFacebook - https://bit.ly/3o6alLt\nInstagram - https://bit.ly/2RFkL8z\n\nFollow my personal handles here: (Chef Sanjyot Keer) \nFacebook - https://bit.ly/3tpjqQA\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\nInstagram - https://bit.ly/3o0FZK8\nTwitter  -  https://bit.ly/33qQpJN\n\nIntro 0:00\nRasam 1:31\nVada 6:50\nPlating 12:50\nOutro 13:55"

```
we can extract things like 
recipe_name, ingredents, preparation_time, cooking_time by passing this description to a LLM model and asking it to return it into JSON format which is then parsed, formatted and validated through pydantic


In [ ]:
import ollama
response = ollama.chat(model='llama3', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])

In [ ]:
import pandas as pd
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List
import google.generativeai as genai
from api_keys import GEMINI_API_KEY

genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash')


class RecipeInfo(BaseModel):
    recipe_name: str = Field(description="Name of the recipe (in English), concise like Dal bati, Red Curry, Rajma Chawal, Rasam wada")
    ingredents: dict = Field(description="Combined single key:value based dict of vegetables,pulses,meat used with quantities like {{'TOMATOES':4,'CHICKEN':'500g'}} ")
    preparation_time: int = Field(description="Preparation time in minutes like 15, 25, 40")
    cooking_time: int = Field(description="Cooking time in minutes like 10, 45, 1hr should be 60")

parser = PydanticOutputParser(pydantic_object=RecipeInfo)
prompt_template = PromptTemplate(
    template=
    """Extract the following information in JSON formal: \n{format_instructions}\n\n
    from the below text: {text}. \n\n
    Links are not required to be extracted. \n\n Return in JSON object only.
    """,
    input_variables=["text"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)


def query_gemini(prompt):
    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"Error calling Gemini API: {e}")
        return None  # Or return a specific error message


def process_recipe(text):
    # Use Gemini instead of Ollama
    prompt = prompt_template.format(text=text)
    response = query_gemini(prompt)
    # print(response)
    if response:
        try:
            parsed_response = parser.parse(response)
            return parsed_response
        except Exception as e:
            print(f"Error parsing response: {e}")
            return None  # Or return a specific error message
    else:
        return None  # Handle no response from Gemini


In [ ]:
test_sample_str = data["description"][1]
x = process_recipe(test_sample_str)

In [ ]:
print(x.recipe_name)
print(x.ingredents)
print(x.preparation_time)
print(x.cooking_time)

In [ ]:
new_cols = ["recipe_name", "ingredients", "preparation_time", "cooking_time"]

def extract_recipe_info(text):
    parsed_data = process_recipe(text)
    if parsed_data:
        return [parsed_data.recipe_name, [parsed_data.ingredents], parsed_data.preparation_time, parsed_data.cooking_time]
    else:
        return [None] * len(new_cols)  # Return a list of None for all columns if parsing fails

In [ ]:
df = data[:5]


for col in new_cols:
    df[col] = None

for i,row in  df.iterrows():
    if df.loc[i, "recipe_name"] is None:
        x = extract_recipe_info(row["description"])
        for j, col in enumerate(new_cols):
            df.loc[i, j] = x[j]
    
    


In [ ]:
df = pd.read_csv("channel_data_parsed_g.csv")
df[20:30]

In [ ]:
df[['recipe_name', 'ingredients', 'preparation_time', 'cooking_time']].head()

## Random observations

In [3]:
import pandas as pd

In [8]:
data = pd.read_csv("channel_data_parsed_g.csv")

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1457 entries, 0 to 1456
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title             1457 non-null   object 
 1   description       1380 non-null   object 
 2   url               1457 non-null   object 
 3   recipe_name       1282 non-null   object 
 4   ingredients       1282 non-null   object 
 5   preparation_time  1282 non-null   float64
 6   cooking_time      1282 non-null   float64
dtypes: float64(2), object(5)
memory usage: 79.8+ KB


In [10]:
data.head()

,title,description,url,recipe_name,ingredients,preparation_time,cooking_time
0,Rasam Vada Recipe | Tomato Rasam | Perfect Vad...,Rasam Powder: https://youtu.be/YIWBthqArCM?t=5...,https://www.youtube.com/watch?v=7LpEu6rTyHs,Rasam Vada,"{'TUR DAL': '0.25', 'WATER': 'AS REQUIRED', 'T...",20.0,30.0
1,Peri Peri Chicken with Butter Rice | Spicy Hom...,Full written recipe of Chicken in Peri Peri Sa...,https://www.youtube.com/watch?v=d9_YSS1lB1w,Chicken in Peri Peri Sauce with Butter Rice,"{'CHICKEN': '400-450 GRAMS', 'OLIVE OIL': '3 T...",15.0,30.0
2,Homestyle Turai Ki Sabzi | Turai Ka Bharta | त...,Full written recipe of Turai Ki Sabzi\nPrep ti...,https://www.youtube.com/watch?v=nX5vogjw_yk,Turai Ki Sabzi,"{'SPONGE GOURD': 1, 'ONION': 2, 'TOMATO': 2, '...",10.0,25.0
3,Chicken Pepper Fry Recipe | Pepper Chicken Mas...,Full written recipe for Chicken Pepper Fry\nPr...,https://www.youtube.com/watch?v=5R6nNJC-FVE,Chicken Pepper Fry,"{'BLACK PEPPERCORNS': 2, 'CUMIN SEEDS': 1, 'FE...",10.0,20.0
4,Achari Paneer Recipe | Dhaba Style Paneer Grav...,Full written recipe of Achari Paneer \nPrep ti...,https://www.youtube.com/watch?v=P514UYjzZ-E,Achari Paneer,"{'FENNEL SEEDS': 2, 'CUMIN SEEDS': 1, 'MUSTARD...",25.0,45.0


In [12]:
data["recipe_id"] = data.index

In [24]:
import ast

data.dropna(subset=["ingredients"], inplace=True)
data["ingredients_dict"] = data["ingredients"].apply(ast.literal_eval)
# data["ingredients"][0]

In [29]:
ings = set()
for i in data["ingredients_dict"]:
    ings.update(i.keys())

In [31]:
len(ings)

2000

In [32]:
ings_df = pd.DataFrame(ings, columns=["ingredient"])

ings_df['recipe_ids'] = ings_df['ingredient'].apply(lambda x: data[data["ingredients_dict"].apply(lambda y: x in y)].index.tolist())

In [35]:
ings_df[ings_df["ingredient"] == "TOMATOES"]

,ingredient,recipe_ids
175,TOMATOES,"[40, 41, 53, 65, 70, 118, 121, 142, 154, 157, ..."


In [36]:
ings_df.to_csv("ingredients.csv", index=False)

In [ ]:
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""
    binary_score: str = Field(description="Documents are relevant to the question, 'yes' or 'no'")


llm = OllamaFunctions(model="gemma:2b", format="json", temperature=0)
structured_llm_documents_grader = llm.with_structured_output(
      GradeDocuments)
chain = grade_prompt | structured_llm_documents_grader


chain.invoke({"question": question, "document": document.page_content})